In [53]:
# STUDENT_ROLLNO = "xxx" #yourRollNumberHere
# STUDENT_NAME = "xxx" #yourNameHere
#@PROTECTED_1
##DO NOT MODIFY THE BELOW CODE. NO OTHER IMPORTS ALLOWED. NO OTHER FILE LOADING OR SAVING ALLOWED.
from torch import Tensor
import torch.nn as nn 
import torch.optim as optim 
import torchmetrics
import sklearn.preprocessing as preprocessing
import sklearn.model_selection as model_selection 
import torch.utils.data as data 
import numpy as np 
import torch
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")
X_test = np.load("X_test.npy")
submission = np.load("sample_submission.npy")
#@PROTECTED_1

#Protected_2 has been moved to the end unedited

In [54]:
X_train = X_train/255.0
X_test = X_test/255.0

In [55]:
X_test = torch.from_numpy(X_test).cuda().float()
X_train = torch.from_numpy(X_train).cuda().float()
y_train = torch.from_numpy(y_train).cuda().float()

In [56]:
class Data(data.Dataset):
    def __init__(self, X, Y) -> None:
        super().__init__()
        self.X = X
        self.Y = Y
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        _x = self.X[index]
        _y = self.Y[index]
        
        return _x, _y


In [57]:
model = nn.Sequential(
    nn.Linear(3072,1024),
    nn.ELU(alpha=1.5),
    nn.Dropout(0.25),

    nn.Linear(1024,512),
    nn.ELU(alpha=1.5),
    nn.Dropout(0.25),

    nn.Linear(512,256),
    nn.ELU(alpha=1.5),
    nn.Dropout(0.25),

    nn.Linear(256,128),
    nn.ELU(alpha=1.5),
    nn.Dropout(0.25),

    nn.Linear(128,64),
    nn.ELU(alpha=1.5),
    nn.Dropout(0.25),
    
    nn.Linear(64,32),
    nn.ELU(alpha=1.5),
    nn.Dropout(0.25),
    nn.Linear(32, 10),
)

In [58]:
cifarrr = model.to('cuda') #converting to tensor
adam_optimizer = optim.Adam(cifarrr.parameters(),lr=0.0001)
cost_function = nn.CrossEntropyLoss().cuda()

system = "cuda" if torch.cuda.is_available() else "cpu"
print("[INFO] training using {}...".format(system))

[INFO] training using cuda...


In [59]:
number_of_epochs = 250
for epoch in tqdm(range(number_of_epochs)):
    cifarrr.train()
    dat_load = data.DataLoader(Data(X_train, y_train), batch_size = 1000, shuffle = True)
    for i, batch in enumerate(dat_load, 0):
        (for_X, for_Y) = (batch[0], batch[1])
        (for_X, for_Y) = (for_X.to(system), for_Y.to(system))
        preds = cifarrr(for_X)
        entropy_loss = cost_function(preds, for_Y.long())
        adam_optimizer.zero_grad()
        entropy_loss.backward()
        adam_optimizer.step()

100%|██████████| 250/250 [01:55<00:00,  2.16it/s]


In [60]:
cifarrr.eval()

Sequential(
  (0): Linear(in_features=3072, out_features=1024, bias=True)
  (1): ELU(alpha=1.5)
  (2): Dropout(p=0.25, inplace=False)
  (3): Linear(in_features=1024, out_features=512, bias=True)
  (4): ELU(alpha=1.5)
  (5): Dropout(p=0.25, inplace=False)
  (6): Linear(in_features=512, out_features=256, bias=True)
  (7): ELU(alpha=1.5)
  (8): Dropout(p=0.25, inplace=False)
  (9): Linear(in_features=256, out_features=128, bias=True)
  (10): ELU(alpha=1.5)
  (11): Dropout(p=0.25, inplace=False)
  (12): Linear(in_features=128, out_features=64, bias=True)
  (13): ELU(alpha=1.5)
  (14): Dropout(p=0.25, inplace=False)
  (15): Linear(in_features=64, out_features=32, bias=True)
  (16): ELU(alpha=1.5)
  (17): Dropout(p=0.25, inplace=False)
  (18): Linear(in_features=32, out_features=10, bias=True)
)

In [61]:
y_pred = cifarrr(X_train)
y_pred = torch.argmax(y_pred, axis=1)

In [62]:
acc = torchmetrics.Accuracy(task="multiclass", num_classes=10, top_k = 1).to('cuda')
acc(y_train, y_pred).cpu().numpy().item()

0.6619399785995483

In [63]:
# #@PROTECTED_2
# np.save("{}__{}".format(STUDENT_ROLLNO,STUDENT_NAME),submission)
# #@PROTECTED_2